# Capítulo 05 — Pré-Treinamento: O Nascimento da Inteligência

Neste capítulo, vamos tirar o nosso GPTMini da inércia. Vamos alimentá-lo com dados e observar o momento exato em que ele deixa de escolher palavras aleatórias e começa a entender os padrões da nossa língua.

--- 
### 🏭 A Fábrica de Predição
O pré-treinamento é a etapa mais pesada. É aqui que o modelo aprende a gramática, o vocabulário e a lógica do mundo através da predição do próximo token.

![Pipeline de Treinamento](./infograficos/01-pipeline-treinamento.png)

In [ ]:
# ============================================================
# Setup do repositório no Colab
# ============================================================
import os, sys
REPO_NAME = "fazendo-um-llm-do-zero"
if 'google.colab' in str(get_ipython()):
    if not os.path.exists(REPO_NAME):
        os.system(f"git clone https://github.com/vongrossi/{REPO_NAME}.git")
    if os.path.exists(REPO_NAME) and os.getcwd().split('/')[-1] != REPO_NAME:
        os.chdir(REPO_NAME)
if os.getcwd() not in sys.path: sys.path.append(os.getcwd())
print("📂 Diretório atual:", os.getcwd())

In [ ]:
!pip -q install -r 05-pre-treinamento/requirements.txt

import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
import math, random, numpy as np
from lib.gptmini import GPTConfig, GPTMini

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"✅ Oficina pronta! Usando: {device}")

torch.manual_seed(42)
np.random.seed(42)
random.seed(42)

## 1. Preparação do Dataset

Vamos usar um conjunto de frases curtas para que o treinamento seja rápido e didático.

In [ ]:
text = """
o gato subiu no telhado
o cachorro subiu no sofa
o gato dormiu no sofa
o cachorro dormiu no tapete
o gato pulou no muro
o cachorro pulou o portao
""".strip().lower()

tokens = text.split()
vocab = sorted(set(tokens))
stoi = {t:i for i,t in enumerate(vocab)}
itos = {i:t for t,i in stoi.items()}
vocab_size = len(vocab)
encoded = [stoi[t] for t in tokens]

def build_dataset(token_ids, context_size=5):
    X, Y = [], []
    for i in range(len(token_ids) - context_size):
        X.append(token_ids[i : i + context_size])
        Y.append(token_ids[i + 1 : i + context_size + 1])
    return torch.tensor(X).to(device), torch.tensor(Y).to(device)

context_size = 5
X, Y = build_dataset(encoded, context_size)
print(f"Vocabulário: {vocab_size} palavras | Amostras: {len(X)}")

## 2. Otimização e Treinamento

Agora vamos instanciar o modelo e treinar usando a **Cross Entropy** em toda a sequência.

![Loop Treino](./infograficos/03-loop-treinamento.png)

In [ ]:
config = GPTConfig(vocab_size=vocab_size, context_size=context_size, d_model=64, n_heads=4, n_layers=2)
model = GPTMini(config).to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3)

train_loss_history = []
print("🚀 Iniciando Treinamento...")
model.train()
for step in range(601):
    idx = torch.randint(0, X.size(0), (16,))
    xb, yb = X[idx], Y[idx]
    logits, loss = model(xb, yb)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    train_loss_history.append(loss.item())
    if step % 100 == 0: print(f"Step {step:03d} | Loss: {loss.item():.4f}")

plt.figure(figsize=(8, 3))
plt.plot(train_loss_history, color='#1A73E8')
plt.title("Curva de Aprendizado (Cross Entropy Loss)")
plt.show()

## 3. Estratégias de Geração (Decoding)

Aqui definimos as funções que permitem ao modelo escolher os próximos tokens.

![Decoding Strategies](./infograficos/04-decoding-strategies.png)

In [ ]:
def encode_text(s): return [stoi[t] for t in s.lower().split() if t in stoi]
def decode(ids): return " ".join(itos[int(i)] for i in ids)

@torch.no_grad()
def generate(model, start_str, max_new=10):
    model.eval()
    idx = torch.tensor(encode_text(start_str)).unsqueeze(0).to(device)
    for _ in range(max_new):
        logits, _ = model(idx[:, -context_size:])
        next_id = torch.argmax(logits[:, -1, :], dim=-1, keepdim=True)
        idx = torch.cat([idx, next_id], dim=1)
    return decode(idx[0].tolist())

print("✨ TESTE DE GERAÇÃO:")
print(f"Entrada: 'o gato' -> Saída: '{generate(model, 'o gato')}'")

## 4. Persistência: Salvando a Inteligência

**O arquivo `gpt_checkpoint.pt` é o "cérebro" que acabamos de treinar.** 

Você precisará dele para os próximos passos (Capítulos 06 e 07).

![Checkpoints](./infograficos/05-checkpoints.png)

In [ ]:
checkpoint = {
    "state_dict": model.state_dict(),
    "stoi": stoi,
    "itos": itos,
    "config": config
}
torch.save(checkpoint, "gpt_checkpoint.pt")
print("✅ Checkpoint salvo com sucesso!")

if 'google.colab' in str(get_ipython()):
    from google.colab import files
    files.download("gpt_checkpoint.pt")

## 🏁 Conclusão

Você acabou de ensinar um GPT a aprender linguagem. Viu como ele sai do caos aleatório para a ordem gramatical e como podemos controlar sua criatividade.

No próximo capítulo, vamos levar este modelo para a **especialização**, aprendendo como fazer o **Fine-Tuning** para tarefas específicas como classificação de sentimentos e detecção de spam.